In [3]:
import pymc as pm
import numpy as np
import arviz as az

%load_ext lab_black
%load_ext watermark

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black
The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark


# Meta-analysis via hierarchical models

This example demonstrates ...

Adapted from [unit 7: rats_nocentering.odc](https://raw.githubusercontent.com/areding/6420-pymc/main/original_examples/Codes4Unit7/rats_nocentering.odc).

Data for x can be found [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/rats_nocenteringx.txt), and Y [here](https://raw.githubusercontent.com/areding/6420-pymc/main/data/rats_nocenteringY.txt).

## Associated lecture video: Unit 7 Lesson 5

In [2]:
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed?v=xomK4tcePmc&list=PLv0FeK5oXK4l-RdT6DWJj0_upJOG2WKNO&index=67" frameborder="0" allow="autoplay; encrypted-media" allowfullscreen></iframe>

In [16]:
Y = np.loadtxt("../data/rats_nocenteringY.txt")
x = np.loadtxt("../data/rats_nocenteringx.txt")

## Problem statement

This example is taken from Gelfand et al (1990), and concerns 30 young rats whose weights were measured weekly for five weeks. Part of the data is shown below, where Yij is the weight of the ith rat measured at age xj.   

need to figure out how to make the shapes equivalent. take a close look at the loops

In [19]:
with pm.Model() as m:
    mu_alpha = pm.Normal("mu_alpha", 0, tau=1e-6)
    tau_alpha = pm.Gamma("tau_alpha", .001, .001)
    mu_beta = pm.Normal("mu_beta", 0, tau=1e-6)
    tau_beta = pm.Gamma("tau_beta", .001, .001)
    
    tau = pm.Gamma("tau", .001, .001)
    
    alpha = pm.Normal("alpha", mu_alpha, tau=tau_alpha, shape=30)
    beta = pm.Normal("beta", mu_beta, tau=tau_beta, shape=30)
    
    mu = alpha + beta * x
    pm.Normal("Y", mu, tau=tau, observed=Y)
    
    trace = pm.sample(3000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...


ValueError: Input dimension mismatch. (input[0].shape[1] = 30, input[2].shape[1] = 5)
Apply node that caused the error: Elemwise{Composite{(i0 + (i1 * i2))}}(InplaceDimShuffle{x,0}.0, InplaceDimShuffle{x,0}.0, TensorConstant{[[ 8. 15. .. 29. 36.]]})
Toposort index: 5
Inputs types: [TensorType(float64, row), TensorType(float64, row), TensorType(float64, row)]
Inputs shapes: [(1, 30), (1, 30), (1, 5)]
Inputs strides: [(240, 8), (240, 8), (40, 8)]
Inputs values: ['not shown', 'not shown', array([[ 8., 15., 22., 29., 36.]])]
Outputs clients: [[Elemwise{Composite{Switch(i0, ((i1 + (i2 * sqr(((i3 - i4) / i5)))) - i6), i7)}}(Elemwise{gt,no_inplace}.0, TensorConstant{(1, 1) of ..5332046727}, TensorConstant{(1, 1) of -0.5}, TensorConstant{[[151. 199..86. 324.]]}, Elemwise{Composite{(i0 + (i1 * i2))}}.0, Elemwise{Composite{reciprocal(sqrt(Switch(i0, i1, i2)))}}[(0, 1)].0, Elemwise{log,no_inplace}.0, TensorConstant{(1, 1) of -inf})]]

HINT: Re-running with most Aesara optimizations disabled could

In [ ]:
az.summary(trace)